In [3]:
import pandas as pd
import numpy as np
import matplotlib
import os

In [210]:
df_train = pd.read_csv("/Users/zamaletdinovaaa/mipt_cet_hack/Silicondzor/data/train.csv", "|")
df_train_sorted = df_train.sort_values(by=['well id', 'depth, m'])
df_train_sorted.to_csv('train_sorted.csv',index=False, sep='|')

In [217]:
df_train_clean = df_train.drop_duplicates(subset=['well id','depth, m','bk','GZ1','GZ2','GZ3','GZ4','GZ5','GZ7','DGK','NKTD','NKTM','NKTR','ALPS'], keep=False)
df_train_clean.to_csv('train_clean.csv',index=False, sep='|')
df_train_clean_sorted = df_train_clean.sort_values(by=['well id', 'depth, m'])
df_train_clean_sorted.to_csv('train_clean_sorted.csv',index=False, sep='|')

In [4]:
df_test = pd.read_csv("/Users/zamaletdinovaaa/mipt_cet_hack/Silicondzor/data/test.csv", "|")

In [ ]:
df_test.

In [169]:
_PNB = 4150
_PHI = 0.7
_RHO = 860
_S = 100
_H_DELTA = 0.1
_EXPENCES_BK = 2450
_EXPENCES_GZ = 2050
_EXPENCES_DGK = 1300
_EXPENCES_NKT = 2050
_EXPENCES_ALPS = 1150
_EXPENCES_DICT = {'bk':_EXPENCES_BK, 
                 'GZ1':_EXPENCES_GZ,
                  'GZ2':_EXPENCES_GZ,
                  'GZ3':_EXPENCES_GZ,
                  'GZ4':_EXPENCES_GZ,
                  'GZ5':_EXPENCES_GZ,
                  'GZ6':_EXPENCES_GZ,
                  'GZ7':_EXPENCES_GZ,
                  'DKG':_EXPENCES_DGK,
                  'NKTD':_EXPENCES_NKT,
                  'NKTM':_EXPENCES_NKT,
                  'NKTR':_EXPENCES_NKT,
                  'ALPS':_EXPENCES_ALPS
                 }
_EXPENCES_COLUMNS_ALL = list(_EXPENCES_DICT.keys())

In [153]:
def analysis_columns(df):
    return list(pd.Index(_EXPENCES_COLUMNS_ALL).intersection(df.columns))
    
def add_expences(df):
    df['expences'] = (df[analysis_columns(df)].fillna(0) * [_EXPENCES_DICT.get(key) for key in analysis_columns(df)]).sum(axis=1)
    return df

In [167]:
def form_result_df(df):
    df_grouped = df.groupby(['well id'])
    columns = ['well id', 'total_cnt', 'oil_cnt', 'value', 'expenses', 'square', 'mass', 'cost']

    df_dashbord_show = pd.DataFrame(columns=columns)
    
    for well_id, df_well in df_grouped:
        total_cnt = df_well.shape[0]
        oil_cnt = df_well[df_well['goal']==1]['well id'].count()
        h = _H_DELTA * oil_cnt
        expences = df_well['expences'].sum()
        square = oil_cnt*_S
        mass = _H_DELTA * square * _RHO
        value = _PNB*_PHI*h*_S*_RHO - expences
        cost = mass * _PNB
        row_dict = [{'cost':cost,'mass':mass,'square':square,'well id':well_id, 'total_cnt':total_cnt, 'oil_cnt':oil_cnt, 'value':value, 'expenses':expences}]
        df_dashbord_show_inc = pd.DataFrame(row_dict)
        df_dashbord_show = df_dashbord_show.append(df_dashbord_show_inc)
    return df_dashbord_show

In [154]:
df_train = add_expences(df_train)    

In [156]:
df_test = add_expences(df_train)    

In [170]:
df_train_result = form_result_df(df_train)

In [181]:
df_train_result.head()

,cost,expenses,mass,oil_cnt,square,total_cnt,value,well id
0,6.281440e+09,6.261313e+06,1513600.0,176,17600,696,4.390747e+09,1.0
0,4.496940e+09,1.342072e+07,1083600.0,126,12600,1472,3.134437e+09,2.0
0,4.068660e+09,1.095490e+07,980400.0,114,11400,1167,2.837107e+09,3.0
0,4.532630e+09,1.066882e+07,1092200.0,127,12700,1170,3.162172e+09,4.0
0,1.070700e+10,1.685752e+07,2580000.0,300,30000,1844,7.478042e+09,5.0


In [10]:
df1[df1.duplicated(['depth, m','bk','GZ1','GZ2','GZ3','GZ4','GZ5','GZ7','DGK','NKTD','NKTM','NKTR','ALPS',], keep=False)]

,id,well id,"depth, m",bk,GZ1,GZ2,GZ3,GZ4,GZ5,GZ7,DGK,NKTD,NKTM,NKTR,ALPS


In [11]:
df_grouped = df_test.groupby(['well id'])

for well_id, df_well in df_grouped:
    print(f'well_id {well_id} nunique - {df_well["depth, m"].nunique()} shape {df_well.shape}')
          
          
df_test[df_test.duplicated(['well id','depth, m','bk','GZ1','GZ2','GZ3','GZ4','GZ5','GZ7','DGK','NKTD','NKTM','NKTR','ALPS',], keep=False)]

well_id 47 nunique - 1297 shape (1360, 15)
well_id 95 nunique - 1162 shape (1199, 15)
well_id 126 nunique - 17382 shape (17633, 15)
well_id 164 nunique - 15826 shape (16238, 15)
well_id 176 nunique - 1136 shape (1174, 15)


,id,well id,"depth, m",bk,GZ1,GZ2,GZ3,GZ4,GZ5,GZ7,DGK,NKTD,NKTM,NKTR,ALPS
30,31,47,2468.231,0.449133,0.376967,0.127989,0.531776,0.253316,0.911877,0.417324,0.769427,0.381311,0.558993,0.422383,0.774000
31,32,47,2468.231,0.449133,0.376967,0.127989,0.531776,0.253316,0.911877,0.417324,0.769427,0.381311,0.558993,0.422383,0.774000
39,40,47,2468.947,0.458829,0.316576,0.117564,0.435454,0.354642,0.893757,0.369689,0.515214,0.381458,0.543162,0.411672,0.774000
40,41,47,2468.947,0.458829,0.316576,0.117564,0.435454,0.354642,0.893757,0.369689,0.515214,0.381458,0.543162,0.411672,0.774000
60,61,47,2470.737,0.367247,0.276679,0.119591,0.552777,0.114990,0.223684,0.645786,0.366495,0.386175,0.563918,0.425497,0.368554
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37520,37521,176,2165.005,0.399551,0.262832,0.127844,0.782174,0.141857,0.368494,0.852176,0.494653,0.347186,0.498485,0.424384,0.162172
37528,37529,176,2165.721,0.423699,0.247487,0.131655,0.675283,0.125893,0.371813,0.841864,0.412178,0.363219,0.513265,0.496400,0.132560
37529,37530,176,2165.721,0.423699,0.247487,0.131655,0.675283,0.125893,0.371813,0.841864,0.412178,0.363219,0.513265,0.496400,0.132560
37565,37566,176,2168.943,0.397237,0.279274,0.154195,0.733595,0.117446,0.357766,0.937225,0.344939,0.393485,0.527332,0.379199,0.195392


In [204]:
df2['well id'].isna().count()

21881

In [190]:
df1=df

In [200]:
df2=df1[df1.duplicated(['well id','depth, m','bk','GZ1','GZ2','GZ3','GZ4','GZ5','GZ7','DGK','NKTD','NKTM','NKTR','ALPS'], keep=False)]




In [ ]:
df.drop_duplicates(subset=['well id','depth, m','bk','GZ1','GZ2','GZ3','GZ4','GZ5','GZ7','DGK','NKTD','NKTM','NKTR','ALPS'], keep=False)